<a href="https://colab.research.google.com/github/manashpratim/Bosch-Summer-Internship/blob/master/Audio_Features_MELD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook is for extracting audio features of MELD dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!unzip -q '/content/drive/My Drive/train_meld.zip'
!unzip -q '/content/drive/My Drive/dev_meld.zip'
#!unzip -q '/content/drive/My Drive/test_meld.zip'

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
#df=pd.read_csv('/content/train_meld/train_sent_emo.csv')
df=pd.read_csv('/content/dev_meld/dev_sent_emo.csv')
#df=pd.read_csv('/content/test_meld/test_sent_emo.csv')

In [ ]:
df.head(5)

In [ ]:
def get_audio_features(mypath,file):
    import librosa
    import numpy as np
    try:
      audiofile = mypath + '/' + file + '.wav'
      x, sr = librosa.load(audiofile, sr  = 16000, res_type='kaiser_fast')        #Sampling frequency 16KHz
      chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr).T
      rmse = librosa.feature.rmse(y=x).T
      spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr).T
      spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr).T
      rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr).T
      zcr = librosa.feature.zero_crossing_rate(x).T
      mfcc = librosa.feature.mfcc(y=x, sr=sr,n_mfcc=40).T                         #40 MFCC features
      myfile = np.hstack((chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc))
      return myfile
    except:
        return []

In [ ]:
path = '/content/dev_meld/audio'
audio = {}
outer = tqdm(total=len(df), desc='Extracting Audio Features', position=0)
for i in range(len(df)):
    outer.update(1)
    name = 'dia'+ str(df['Dialogue_ID'][i]) + '_utt' + str(df['Utterance_ID'][i])
    aud = get_audio_features(path,name)
    if len(aud)>0:
      audio[name] = aud

In [ ]:
import pickle
with open('/content/drive/My Drive/meld_data/audio_features_dev.pickle', 'wb') as handle:
        pickle.dump(audio, handle, protocol=pickle.HIGHEST_PROTOCOL)